# BAH_TheGuild final submission - Public .538 Private .532

## Data Science Bowl 2019

In [1]:
# Cleaned up redundant or unused code from Seals-final.ipynb final submission. See that notebook for full executed notebook

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [3]:
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from collections import Counter
#import mlflow              #MLFlow only used locally for tracking test results
import sklearn
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 


Using TensorFlow backend.
C:\Users\604572\Continuum\anaconda3\envs\dsb2019\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\604572\Continuum\anaconda3\envs\dsb2019\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\604572\Continuum\anaconda3\envs\dsb2019\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users

In [4]:
from lightgbm import LGBMRegressor
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
)
from catboost import CatBoostRegressor
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.svm import LinearSVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import (
    LogisticRegression,
    SGDRegressor,
    Ridge,
    LinearRegression,
    Lasso
)
import warnings

warnings.filterwarnings(
    "ignore"
)  


In [7]:
# Any results you write to the current directory are saved as output.

# %% [code]
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor
from xgboost import plot_importance
from catboost import CatBoostRegressor
from matplotlib import pyplot
import shap
import random

import os

for dirname, _, filenames in os.walk("/kaggle/input"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
from time import time
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from scipy import stats
import lightgbm as lgb
from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
import gc
import json
import joblib

pd.set_option("display.max_columns", 1000)

# %% [markdown]
# # Notes
# * Check the distribution of the target variable of the out of folds score and the prediction distribution. A good model should more or less have the same distribution.

# %% [code]
def read_data():    ## currently pointing to local directories
    print("Reading train.csv file....")
    train = pd.read_csv("data/train.csv")
    print(
        "Training.csv file have {} rows and {} columns".format(
            train.shape[0], train.shape[1]
        )
    )

    print("Reading test.csv file....")
    test = pd.read_csv("data/test.csv")
    print(
        "Test.csv file have {} rows and {} columns".format(test.shape[0], test.shape[1])
    )

    print("Reading train_labels.csv file....")
    train_labels = pd.read_csv("data/train_labels.csv")
    print(
        "Train_labels.csv file have {} rows and {} columns".format(
            train_labels.shape[0], train_labels.shape[1]
        )
    )

    return train, test, train_labels


# %% [code]
def encode_title(train, test, train_labels):
    # encode title
    train["title_event_code"] = list(
        map(lambda x, y: str(x) + "_" + str(y), train["title"], train["event_code"])
    )
    test["title_event_code"] = list(
        map(lambda x, y: str(x) + "_" + str(y), test["title"], test["event_code"])
    )
    all_title_event_code = list(
        set(train["title_event_code"].unique()).union(test["title_event_code"].unique())
    )
    # make a list with all the unique 'titles' from the train and test set
    list_of_user_activities = list(
        set(train["title"].unique()).union(set(test["title"].unique()))
    )
    # make a list with all the unique 'event_code' from the train and test set
    list_of_event_code = list(
        set(train["event_code"].unique()).union(set(test["event_code"].unique()))
    )
    list_of_event_id = list(
        set(train["event_id"].unique()).union(set(test["event_id"].unique()))
    )
    # make a list with all the unique worlds from the train and test set
    list_of_worlds = list(
        set(train["world"].unique()).union(set(test["world"].unique()))
    )
    # create a dictionary numerating the titles
    activities_map = dict(
        zip(list_of_user_activities, np.arange(len(list_of_user_activities)))
    )
    activities_labels = dict(
        zip(np.arange(len(list_of_user_activities)), list_of_user_activities)
    )
    activities_world = dict(zip(list_of_worlds, np.arange(len(list_of_worlds))))
    assess_titles = list(
        set(train[train["type"] == "Assessment"]["title"].value_counts().index).union(
            set(test[test["type"] == "Assessment"]["title"].value_counts().index)
        )
    )
    # replace the text titles with the number titles from the dict
    train["title"] = train["title"].map(activities_map)
    test["title"] = test["title"].map(activities_map)
    train["world"] = train["world"].map(activities_world)
    test["world"] = test["world"].map(activities_world)
    train_labels["title"] = train_labels["title"].map(activities_map)
    win_code = dict(
        zip(
            activities_map.values(), (4100 * np.ones(len(activities_map))).astype("int")
        )
    )
    # then, it set one element, the 'Bird Measurer (Assessment)' as 4110, 10 more than the rest
    win_code[activities_map["Bird Measurer (Assessment)"]] = 4110
    # convert text into datetime
    train["timestamp"] = pd.to_datetime(train["timestamp"])
    test["timestamp"] = pd.to_datetime(test["timestamp"])

    return (
        train,
        test,
        train_labels,
        win_code,
        list_of_user_activities,
        list_of_event_code,
        activities_labels,
        assess_titles,
        list_of_event_id,
        all_title_event_code,
    )


# %% [code]
# this is the function that convert the raw data into processed features
def get_data(user_sample, test_set=False):
    """
    The user_sample is a DataFrame from train or test where the only one 
    installation_id is filtered
    And the test_set parameter is related with the labels processing, that is only requered
    if test_set=False
    """
    # Constants and parameters declaration
    last_activity = 0

    user_activities_count = {"Clip": 0, "Activity": 0, "Assessment": 0, "Game": 0}

    # new features: time spent in each activity
    last_session_time_sec = 0
    accuracy_groups = {0: 0, 1: 0, 2: 0, 3: 0}
    all_assessments = []
    accumulated_accuracy_group = 0
    accumulated_accuracy = 0
    accumulated_correct_attempts = 0
    accumulated_uncorrect_attempts = 0
    accumulated_actions = 0
    counter = 0
    time_first_activity = float(user_sample["timestamp"].values[0])
    durations = []
    last_accuracy_title = {"acc_" + title: -1 for title in assess_titles}
    event_code_count: Dict[str, int] = {ev: 0 for ev in list_of_event_code}
    event_id_count: Dict[str, int] = {eve: 0 for eve in list_of_event_id}
    title_count: Dict[str, int] = {eve: 0 for eve in activities_labels.values()}
    title_event_code_count: Dict[str, int] = {
        t_eve: 0 for t_eve in all_title_event_code
    }

    # last features
    sessions_count = 0

    # itarates through each session of one instalation_id
    for i, session in user_sample.groupby("game_session", sort=False):
        # i = game_session_id
        # session is a DataFrame that contain only one game_session

        # get some sessions information
        session_type = session["type"].iloc[0]
        session_title = session["title"].iloc[0]
        session_title_text = activities_labels[session_title]
        game_session = session["game_session"].iloc[0]

        # for each assessment, and only this kind off session, the features below are processed
        # and a register are generated
        if (session_type == "Assessment") & (test_set or len(session) > 1):
            # search for event_code 4100, that represents the assessments trial
            all_attempts = session.query(f"event_code == {win_code[session_title]}")
            # then, check the numbers of wins and the number of losses
            true_attempts = all_attempts["event_data"].str.contains("true").sum()
            false_attempts = all_attempts["event_data"].str.contains("false").sum()
            # copy a dict to use as feature template, it's initialized with some itens:
            # {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
            features = user_activities_count.copy()
            features.update(last_accuracy_title.copy())
            features.update(event_code_count.copy())
            features.update(event_id_count.copy())
            features.update(title_count.copy())
            features.update(title_event_code_count.copy())
            features.update(last_accuracy_title.copy())
            features["installation_session_count"] = sessions_count

            variety_features = [
                ("var_event_code", event_code_count),
                ("var_event_id", event_id_count),
                ("var_title", title_count),
                ("var_title_event_code", title_event_code_count),
            ]

            for name, dict_counts in variety_features:
                arr = np.array(list(dict_counts.values()))
                features[name] = np.count_nonzero(arr)

            # get installation_id for aggregated features
            features["installation_id"] = session["installation_id"].iloc[-1]
            features["game_session"] = game_session
            # add title as feature, remembering that title represents the name of the game
            features["session_title"] = session["title"].iloc[0]
            # the 4 lines below add the feature of the history of the trials of this player
            # this is based on the all time attempts so far, at the moment of this assessment
            features["accumulated_correct_attempts"] = accumulated_correct_attempts
            features["accumulated_uncorrect_attempts"] = accumulated_uncorrect_attempts
            accumulated_correct_attempts += true_attempts
            accumulated_uncorrect_attempts += false_attempts
            # the time spent in the app so far
            if durations == []:
                features["duration_mean"] = 0
                features["duration_std"] = 0
            else:
                features["duration_mean"] = np.mean(durations)
                features["duration_std"] = np.std(durations)
            durations.append((session.iloc[-1, 2] - session.iloc[0, 2]).seconds)
            # the accurace is the all time wins divided by the all time attempts
            features["accumulated_accuracy"] = (
                accumulated_accuracy / counter if counter > 0 else 0
            )
            accuracy = (
                true_attempts / (true_attempts + false_attempts)
                if (true_attempts + false_attempts) != 0
                else 0
            )
            accumulated_accuracy += accuracy
            last_accuracy_title["acc_" + session_title_text] = accuracy
            # a feature of the current accuracy categorized
            # it is a counter of how many times this player was in each accuracy group
            if accuracy == 0:
                features["accuracy_group"] = 0
            elif accuracy == 1:
                features["accuracy_group"] = 3
            elif accuracy == 0.5:
                features["accuracy_group"] = 2
            else:
                features["accuracy_group"] = 1
            features.update(accuracy_groups)
            accuracy_groups[features["accuracy_group"]] += 1
            # mean of the all accuracy groups of this player
            features["accumulated_accuracy_group"] = (
                accumulated_accuracy_group / counter if counter > 0 else 0
            )
            accumulated_accuracy_group += features["accuracy_group"]
            # how many actions the player has done so far, it is initialized as 0 and updated some lines below
            features["accumulated_actions"] = accumulated_actions

            # there are some conditions to allow this features to be inserted in the datasets
            # if it's a test set, all sessions belong to the final dataset
            # it it's a train, needs to be passed throught this clausule: session.query(f'event_code == {win_code[session_title]}')
            # that means, must exist an event_code 4100 or 4110
            if test_set:
                all_assessments.append(features)
            elif true_attempts + false_attempts > 0:
                all_assessments.append(features)

            counter += 1

        sessions_count += 1
        # this piece counts how many actions was made in each event_code so far
        def update_counters(counter: dict, col: str):
            num_of_session_count = Counter(session[col])
            for k in num_of_session_count.keys():
                x = k
                if col == "title":
                    x = activities_labels[k]
                counter[x] += num_of_session_count[k]
            return counter

        event_code_count = update_counters(event_code_count, "event_code")
        event_id_count = update_counters(event_id_count, "event_id")
        title_count = update_counters(title_count, "title")
        title_event_code_count = update_counters(
            title_event_code_count, "title_event_code"
        )

        # counts how many actions the player has done so far, used in the feature of the same name
        accumulated_actions += len(session)
        if last_activity != session_type:
            user_activities_count[session_type] += 1
            last_activitiy = session_type

    # if it't the test_set, only the last assessment must be predicted, the previous are scraped
    if test_set:
        return all_assessments[-1]
    # in the train_set, all assessments goes to the dataset
    return all_assessments


# %% [code]
# Compile the training and testing data 
def get_train_and_test(train, test):
    compiled_train = []
    compiled_test = []
    assessment_sessions_by_instid = {}
    # Loop through each train installation id
    for ins_id, user_sample in tqdm(
        train.groupby("installation_id", sort=False),
        total=train["installation_id"].nunique(),
    ):
        compiled_train += get_data(user_sample, test_set=False)
    reduce_train = pd.DataFrame(compiled_train)
    del compiled_train
    # Loop through each test installation id
    for ins_id, user_sample in tqdm(
        test.groupby("installation_id", sort=False),
        total=test["installation_id"].nunique(),
    ):
        test_data = get_data(user_sample, test_set=True)
        compiled_test.append(test_data)
    reduce_test = pd.DataFrame(compiled_test)
    del compiled_test
    categoricals = ["session_title"]
    return reduce_train, reduce_test, categoricals


# Remove elements that aren't needed
def remove_dead_weight(df, train_labels, test_set=False):
    data_df = pd.DataFrame(df).copy()
    data_df = data_df[data_df.world != "NONE"]

    # Filter out only the installation ids with assessments
    keep_id = data_df[data_df.type == "Assessment"][
        ["installation_id"]
    ].drop_duplicates()
    data_df = pd.merge(data_df, keep_id, on="installation_id", how="inner")

    # Filter out installation ids with more than 4000 event code counts
    #df_grouped = data_df.groupby("installation_id")["event_id"].count()
    #keep_count_ids = df_grouped[df_grouped < 6000]

    #data_df = data_df[data_df.installation_id.isin(keep_count_ids.index)]

    # If training set then make sure the installation ids are in the labels and remove assements not in the labels
    if test_set == False:
        data_df.reset_index()
        data_df = data_df[
            data_df.installation_id.isin(train_labels.installation_id.unique())
        ]
        assessments = data_df[data_df.type == "Assessment"]
        assessments = assessments[
            ~assessments.game_session.isin(train_labels.game_session)
        ]
        data_df = data_df[~data_df.game_session.isin(assessments.game_session)]
        data_df.reset_index()

    return data_df


# %% [code]
# read data
train, test, train_labels = read_data()

# %% [code]
# remove unwanted data
train = remove_dead_weight(train, train_labels, test_set=False)
test = remove_dead_weight(test, train_labels, test_set=True)

# %% [code]
# get usefull dict with maping encode
train, test, train_labels, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, list_of_event_id, all_title_event_code = encode_title(
    train, test, train_labels
)

# %% [code]
# tranform function to get the train and test set
reduce_train, reduce_test, categoricals = get_train_and_test(train, test)

# %% [code]
# Delete train and terst to free up resources
del train
del test

# Save off the train/test data so local testing can skip the long compilation process
reduce_train.to_csv("reduce_train.csv")
reduce_test.to_csv("reduce_test.csv")
joblib.dump(categoricals, 'categoricals.txt')

Reading train.csv file....
Training.csv file have 11341042 rows and 11 columns
Reading test.csv file....
Test.csv file have 1156414 rows and 11 columns
Reading train_labels.csv file....
Train_labels.csv file have 17690 rows and 7 columns


['categoricals.txt']

In [8]:
# Drop alphanumeric and target columns
cols_to_drop = ["game_session", "installation_id", "accuracy_group"]


from sklearn.metrics import make_scorer
kappa_scorer = make_scorer(cohen_kappa_score, weights="quadratic")

from sklearn.metrics import make_scorer, accuracy_score, cohen_kappa_score
from sklearn.model_selection import train_test_split

# get prediction
# this takes regression outputs and rounds them to integers by identifying distribution of accuracy groups in training data
def get_class_pred(pred, train_t):
    """
    Fast cappa eval function for regression outputs
    """
    dist = Counter(train_t["accuracy_group"])
    for k in dist:
        dist[k] /= len(train_t)

    acum = 0
    bound = {}
    for i in range(3):
        acum += dist[i]
        bound[i] = np.percentile(pred, acum * 100)

    def classify(x):
        if x <= bound[0]:
            return 0
        elif x <= bound[1]:
            return 1
        elif x <= bound[2]:
            return 2
        else:
            return 3

    y_pred = np.array(list(map(classify, pred)))

    return y_pred


def make_submission(preds, train_df):
    """ Format final predictions in Kaggle-acceptable submission format. """
    preds = get_class_pred(preds, train_df)
    # assert len(preds)==1000
    sample = pd.read_csv("/kaggle/input/data-science-bowl-2019/sample_submission.csv")
    submission = pd.DataFrame()
    submission["installation_id"] = sample["installation_id"]
    submission["accuracy_group"] = preds
    submission.to_csv("submission.csv", index=False)
    return submission

### Create metric function to use with vecstack - https://github.com/vecxoz/vecstack

In [9]:
def my_metric(y_true, y_pred):
    """ Create function to use with vecstack library - allows for kappa scoring as cross-validation metric"""
    y_pred = get_class_pred(y_pred, reduce_train)
    qwk = cohen_kappa_score(y_true, y_pred, weights='quadratic')
    return qwk

### Instantiate variety of regression models for use in vecstack

In [10]:
lgbm = LGBMRegressor()
rfr = RandomForestRegressor()
cbr = CatBoostRegressor(**{
            'loss_function': 'MultiRMSE',
        'task_type': "CPU",
        'iterations': 1860,
        'depth': 6,
        'early_stopping_rounds': 300,
        'l2_leaf_reg': 2,
        'rsm': 1,
        'bootstrap_type': 'Bayesian',
        'bagging_temperature': 1,
        'random_seed': 42,
        'learning_rate': 0.04,
        'eval_metric': 'MultiRMSE',
        'silent':True}
)
gbr = GradientBoostingRegressor()
abr = AdaBoostRegressor()
lvr = LinearSVR()
lr = LinearRegression()
rr = Ridge()

models = [cbr, lgbm, rfr, gbr, abr, lr, rr]

In [148]:
model_names = ['CatboostReg', 'LightGBMReg', 'RandForReg', 'GradBoostReg', 'AdaBoostReg', 'LinearReg', 'RidgeReg']

In [11]:
from vecstack import stacking

mms = MinMaxScaler()   #scaling wasn't used in final... only seemed to impact a neural net that didn't get used in the end

#  Read in and scale data

# If running for submission use these 3 lines:
# X_train = reduce_train.drop(cols_to_drop, axis=1, errors="ignore")
# y_train = reduce_train.accuracy_group
# X_test = reduce_test.drop(cols_to_drop, axis=1, errors="ignore")

# If testing locally, use these line:
X = reduce_train.drop(cols_to_drop, axis=1, errors='ignore')
y = reduce_train.accuracy_group
X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.2, random_state=42
)

#X_train, X_test = mms.fit_transform(X_train), mms.transform(X_test)

### Create a stack train, and stack test with our initial list of models. Cross-val scores are printed below

In [12]:
S_train, S_test = stacking(
    models,
    X_train,
    y_train,
    X_test,
    regression=True,
    mode="oof_pred_bag",
    needs_proba=False,
    save_dir=None,
    metric=my_metric,
    n_folds=4,
    shuffle=True,
    random_state=42,
    verbose=2,
)

task:         [regression]
metric:       [my_metric]
mode:         [oof_pred_bag]
n_models:     [7]

model  0:     [CatBoostRegressor]
    fold  0:  [0.55146660]
    fold  1:  [0.58909119]
    fold  2:  [0.58220102]
    fold  3:  [0.60615071]
    ----
    MEAN:     [0.58222738] + [0.01978424]
    FULL:     [0.58348677]

model  1:     [LGBMRegressor]
    fold  0:  [0.55164641]
    fold  1:  [0.58392701]
    fold  2:  [0.57631335]
    fold  3:  [0.60934193]
    ----
    MEAN:     [0.58030717] + [0.02057614]
    FULL:     [0.58152061]

model  2:     [RandomForestRegressor]
    fold  0:  [0.53689162]
    fold  1:  [0.56873301]
    fold  2:  [0.55651968]
    fold  3:  [0.58978876]
    ----
    MEAN:     [0.56298327] + [0.01919730]
    FULL:     [0.56200348]

model  3:     [GradientBoostingRegressor]
    fold  0:  [0.54553290]
    fold  1:  [0.58748851]
    fold  2:  [0.56591575]
    fold  3:  [0.60118660]
    ----
    MEAN:     [0.57503094] + [0.02116914]
    FULL:     [0.57486248]

model  

In [97]:
print(S_train[0])
print(y_train)

[0.83004071 0.84457287 0.83       0.99287247 1.39287733 1.56053037
 1.56235911]
7682     0
13165    2
4288     3
9492     1
8651     3
        ..
11284    3
11964    3
5390     3
860      2
15795    3
Name: accuracy_group, Length: 14152, dtype: int64


### Fit final model on S_train, y_train. This second-level model trains on all the former model outputs and creates the final prediction on S_test

In [13]:
model = LinearRegression()

model = model.fit(S_train, y_train)
y_pred = model.predict(S_test)
#y_pred = get_class_pred(y_pred, reduce_train)
#print('Final prediction score: [%.8f]' % my_metric(y_test, y_pred))

### Final submission

In [14]:
#make_submission(y_pred, reduce_train)

FileNotFoundError: [Errno 2] File b'/kaggle/input/data-science-bowl-2019/sample_submission.csv' does not exist: b'/kaggle/input/data-science-bowl-2019/sample_submission.csv'

### Features Imporance Summary

In [23]:
from sklearn.feature_selection import SelectFromModel

In [32]:
X_train_, y_train_ = X_train.iloc[:20, :], y_train.iloc[:20]

In [41]:
X_train.columns[feature_idx].values

array(['acc_Bird Measurer (Assessment)', 4035, 2020, '33505eae',
       '5f0eb72c', 'e9c52111', 'ca11f653', '53c6e11a', 'd02b7a8e',
       'b74258a0', '7cf1bc53', 'c277e121', 'e57dd7af', 'c58186bf',
       '155f62a4', '804ee27f', '37ee8496', '1c178d24', 'e7561dd2',
       '3edf6747', 'b7530680', 'df4940d3', '392e14df',
       'Crystal Caves - Level 1', 'Leaf Leader', 'Leaf Leader_3110',
       'Chow Time_4070', 'Pan Balance_3121', 'Happy Camel_4070',
       'Mushroom Sorter (Assessment)_2025', 'Pan Balance_2030',
       'Rulers_2000', 'Cauldron Filler (Assessment)_4100',
       'Bug Measurer (Activity)_4070',
       'Mushroom Sorter (Assessment)_4020', 'Chow Time_4035',
       'Cauldron Filler (Assessment)_3120', 'All Star Sorting_2020',
       'Crystal Caves - Level 3_2000', 'Dino Dive_4020',
       'Pan Balance_3021', 'Bubble Bath_4040',
       'Egg Dropper (Activity)_4070', 'Chow Time_3020',
       'Cart Balancer (Assessment)_4040', 'Happy Camel_3021',
       'All Star Sorting_3121'

In [48]:
# use feature importance for feature selection
top_features = []
for model in models:
    selection = SelectFromModel(model.fit(X_train, y_train), prefit=True, max_features=50)
    feature_idx = selection.get_support()
    feature_name = list(X_train.columns[feature_idx].values)
    top_features+=feature_name

len(models), len(top_features)

(7, 319)

In [49]:
from collections import Counter

top_feature_counts = Counter(top_features)
print(top_feature_counts.most_common(25))

[('acc_Chest Sorter (Assessment)', 7), ('accumulated_accuracy', 7), ('acc_Bird Measurer (Assessment)', 6), ('Clip', 5), ('acc_Mushroom Sorter (Assessment)', 5), ('acc_Cart Balancer (Assessment)', 5), (4070, 5), ('session_title', 5), ('accumulated_accuracy_group', 5), (3120, 4), (3121, 4), (4020, 4), (3020, 4), (3021, 4), (2000, 4), (2030, 4), (4090, 4), ('7372e1a5', 4), ('27253bdc', 4), ('var_title', 4), ('duration_std', 4), (3110, 3), (4025, 3), (4030, 3), (3010, 3)]


In [93]:
lr = LinearRegression()
lr.fit(X_train, y_train)
selection = SelectFromModel(lr, prefit=True, max_features=50)
selection.estimator_.coef_

AttributeError: 'SelectFromModel' object has no attribute 'estimator_'

In [67]:
corrs = reduce_train[reduce_train.columns[:]].corr()['accuracy_group'][:-1]

In [78]:
type(corrs)
labeled_corrs = list(zip(corrs.index, corrs.values))
labeled_corrs

[('Clip', -0.11402370267541358),
 ('Activity', -0.05516208333194491),
 ('Assessment', -0.06883711773616298),
 ('Game', -0.058998435195202474),
 ('acc_Cauldron Filler (Assessment)', 0.04168309372684425),
 ('acc_Mushroom Sorter (Assessment)', 0.06573486752060995),
 ('acc_Bird Measurer (Assessment)', 0.08863236774736744),
 ('acc_Chest Sorter (Assessment)', 0.08116506017256891),
 ('acc_Cart Balancer (Assessment)', 0.0005848859645687525),
 (2050, -0.008919108525205555),
 (4100, -0.07428824516319973),
 (2060, -0.020625962676432318),
 (4110, -0.04379283385087215),
 (2070, -0.00030423065037610237),
 (2075, -0.027390341730999312),
 (2080, -0.0223261556653702),
 (2081, -0.041000371577051116),
 (2083, -0.005225630614599413),
 (3110, -0.03696782474199145),
 (3120, -0.09415825884176492),
 (3121, -0.004346134950400647),
 (4220, 0.033518485772333066),
 (4230, 0.02179812749190539),
 (5000, 0.005546879519689446),
 (4235, 0.021789813496514567),
 (5010, 0.005280537878085218),
 (4010, -0.05446483313648221),
 (4020, -0.03581494552491916),
 (4021, -0.04477347001757898),
 (4022, -0.018092807105748593),
 (4025, -0.047915707639009576),
 (4030, -0.05862607700326858),
 (4031, -0.01315758667033186),
 (3010, -0.03803935228759694),
 (4035, -0.11028048041822197),
 (4040, -0.038015047156873996),
 (3020, -0.09576027391002949),
 (3021, -0.0045200240475035715),
 (4045, -0.01171716650254826),
 (2000, -0.09379082432209815),
 (4050, -0.01291053897487436),
 (2010, 0.009277161466843485),
 (2020, -0.015585812831609383),
 (4070, -0.09432219721888202),
 (2025, -0.06803080343704565),
 (2030, -0.001742036972421244),
 (4080, -0.04508897484010207),
 (2035, -0.04162116563553152),
 (2040, -0.013887985567473466),
 (4090, -0.13260830418521652),
 (4095, -0.019492599095327027),
 ('77261ab5', -0.07503052748717438),
 ('562cec5f', 0.037061209436155616),
 ('4bb2f698', -0.05220886860634446),
 ('d185d3ea', -0.07426308678795111),
 ('d51b1749', -0.009524361505293852),
 ('763fc34e', -0.020589690198980947),
 ('28a4eb9a', -0.026151820556234776),
 ('30df3273', -0.016161217343912498),
 ('709b1251', 0.024429889264478165),
 ('1bb5fbdb', -0.05919537997835922),
 ('9ed8f6da', -0.024024108744498715),
 ('3a4be871', -0.010828166058721287),
 ('0d1da71f', -0.043500189718190085),
 ('363c86c9', -0.13069772379684363),
 ('022b4259', -0.03694445292797052),
 ('8fee50e2', 0.08081435404198963),
 ('e7e44842', -0.024520736658879844),
 ('3ee399c3', -0.11750032759959078),
 ('1af8be29', -0.041573813263789026),
 ('28f975ea', -0.021023455640203345),
 ('16667cc5', -0.00880535825995438),
 ('1996c610', -0.01315758667033186),
 ('28520915', -0.0458043479098791),
 ('0ce40006', 0.0011068043567499237),
 ('f50fc6c1', -0.018101917739646506),
 ('33505eae', -0.029332610780895003),
 ('6043a2b4', -0.04902117798179915),
 ('cf7638f3', 0.013258631771952509),
 ('fd20ea40', -0.03251742813721296),
 ('d2278a3b', -0.03140873105639236),
 ('19967db1', -0.04332705925002675),
 ('5f0eb72c', -0.10354195844123731),
 ('c2baf0bd', -0.0028260162844655334),
 ('05ad839b', -0.029814959265254105),
 ('4c2ec19f', -0.026043175171139524),
 ('4ef8cdd3', -0.04702789179076167),
 ('d3f1e122', -0.021053207547551578),
 ('74e5f8a7', -0.007430897811895748),
 ('67aa2ada', -0.019122450444128743),
 ('26a5a3dd', -0.01196977992138935),
 ('b80e5e84', -0.03794738785205792),
 ('51311d7a', -0.028931885580231725),
 ('7d5c30a2', -0.006894165219462516),
 ('89aace00', -0.039782433991086275),
 ('9e34ea74', -0.05788985089110455),
 ('5f5b2617', -0.004979506074700979),
 ('9d29771f', -0.06803698960166725),
 ('6c930e6e', -0.017039532404346625),
 ('5d042115', -0.024902312420325347),
 ('0413e89d', -0.01009608359126701),
 ('25fa8af4', -0.1240774806863777),
 ('a1e4395d', -0.08600150905677406),
 ('070a5291', 0.08372382549156465),
 ('08fd73f3', -0.01089724904154084),
 ('f7e47413', -0.014533772221486227),
 ('dcaede90', -0.013887985567473466),
 ('1cf54632', -0.1032764733011338),
 ('895865f3', 0.0011317318911723462),
 ('3323d7e9', 0.00045169661376207363),
 ('884228c8', -0.04729225129046903)

In [83]:
print(sorted(labeled_corrs, reverse=True, key=lambda x: x[1])[:50])

[('accuracy_group', 1.0), ('Bird Measurer (Assessment)_2010', 0.11924716222818339), ('f6947f54', 0.11754375411745538), ('Bird Measurer (Assessment)_2030', 0.11754375411745538), ('7525289a', 0.11177947205766506), ('45d01abe', 0.11066632797294382), ('Bird Measurer (Assessment)_3021', 0.11066632797294382), ('acc_Bird Measurer (Assessment)', 0.08863236774736744), ('070a5291', 0.08372382549156465), ('acc_Chest Sorter (Assessment)', 0.08116506017256891), ('8fee50e2', 0.08081435404198963), ('3afb49e6', 0.06997925389372843), ('e4f1efe6', 0.06985253198784346), ('38074c54', 0.06948306246835842), ('222660ff', 0.06948306246835842), ('acc_Mushroom Sorter (Assessment)', 0.06573486752060995), ('d38c2fd7', 0.05147620702915736), ('acc_Cauldron Filler (Assessment)', 0.04168309372684425), ('562cec5f', 0.037061209436155616), (4220, 0.033518485772333066), ('1340b8d7', 0.033518485772333066), ('a8efe47b', 0.032845246028863566), ('00c73085', 0.031173558157689865), ('b012cd7f', 0.028686242454941214), ('e5c9df6

In [86]:
corr_dataframe = pd.DataFrame(sorted(labeled_corrs, reverse=True, key=lambda x: x[1])[:50], columns=['feature', 'correlation'])

In [91]:
corr_dataframe['Model'] = str(model).split('(')[0]

In [92]:
corr_dataframe

,feature,correlation,Model
0,accuracy_group,1.000000,Ridge
1,Bird Measurer (Assessment)_2010,0.119247,Ridge
2,f6947f54,0.117544,Ridge
3,Bird Measurer (Assessment)_2030,0.117544,Ridge
4,7525289a,0.111779,Ridge
5,45d01abe,0.110666,Ridge
6,Bird Measurer (Assessment)_3021,0.110666,Ridge
7,acc_Bird Measurer (Assessment),0.088632,Ridge
8,070a5291,0.083724,Ridge
9,acc_Chest Sorter (Assessment),0.081165,Ridge


In [202]:
feature_list = []
for model in tqdm(models, total=len(models)):
    model.fit(X_train, y_train)
    try:
        rankings = model.feature_importances_
        rankings = sorted(list(zip(model.feature_importances_, X_train.columns)), key = lambda x:x[0], reverse=True)[:25]
        feature_list.append(rankings)
    except Exception as e:
        print(e)
        print(f'{str(model).split("(")[0]} has no "feature_importances_" method, attempting coef_')
        print('-'*50)
        rankings = model.coef_
        rankings = sorted(list(zip(model.coef_, X_train.columns)), key = lambda x:abs(x[0]), reverse=True)[:25]
        feature_list.append(rankings)

              


'LinearRegression' object has no attribute 'feature_importances_'
LinearRegression has no "feature_importances_" method, attempting coef_
--------------------------------------------------
'Ridge' object has no attribute 'feature_importances_'
Ridge has no "feature_importances_" method, attempting coef_
--------------------------------------------------



In [203]:
i = 0
alldf=[]
for features in feature_list:
    ranked_features = [(features.index(x)+1, x[1]) for x in features]
    df = pd.DataFrame(ranked_features, columns=['Rank', 'Feature'])
    df['Model'] = str(model_names[i])
    alldf.append(df)
    i += 1
    
alldf = pd.concat(alldf)

In [204]:
alldf.to_csv('Top_25_Features.csv', index=False)

In [205]:
alldf.head()

,Rank,Feature,Model
0,1,session_title,CatboostReg
1,2,accumulated_accuracy,CatboostReg
2,3,accumulated_accuracy_group,CatboostReg
3,4,4070,CatboostReg
4,5,duration_mean,CatboostReg


In [206]:
import plotly.express as px

fig = px.parallel_coordinates(alldf, 
                              color=alldf.Model,
                              )


fig.show()

ValueError: 
    Invalid element(s) received for the 'color' property of parcoords.line
        Invalid elements include: ['CatboostReg', 'CatboostReg', 'CatboostReg', 'CatboostReg', 'CatboostReg', 'CatboostReg', 'CatboostReg', 'CatboostReg', 'CatboostReg', 'CatboostReg']

    The 'color' property is a color and may be specified as:
      - A hex string (e.g. '#ff0000')
      - An rgb/rgba string (e.g. 'rgb(255,0,0)')
      - An hsl/hsla string (e.g. 'hsl(0,100%,50%)')
      - An hsv/hsva string (e.g. 'hsv(0,100%,100%)')
      - A named CSS color:
            aliceblue, antiquewhite, aqua, aquamarine, azure,
            beige, bisque, black, blanchedalmond, blue,
            blueviolet, brown, burlywood, cadetblue,
            chartreuse, chocolate, coral, cornflowerblue,
            cornsilk, crimson, cyan, darkblue, darkcyan,
            darkgoldenrod, darkgray, darkgrey, darkgreen,
            darkkhaki, darkmagenta, darkolivegreen, darkorange,
            darkorchid, darkred, darks

In [184]:
all_df_pivoted = alldf.pivot(index='Model', columns='Feature', values='Rank')
all_df_pivoted

Feature,0,2000,2010,2020,2025,2030,2050,2060,2070,2075,2080,2081,2083,3010,3020,3021,3110,3120,3121,4010,4020,4021,4022,4025,4030,4031,4035,4040,4070,4090,4100,4110,4220,4230,4235,5000,5010,01ca3a3c,04df9b66,0ce40006,0db6d71d,1325467d,15a43e5b,1af8be29,1b54d27f,27253bdc,29a42aea,30614231,363c86c9,37db1c2f,37ee8496,392e14df,3afb49e6,3bb91dda,3d8c61b0,3edf6747,3ee399c3,45d01abe,47026d5f,5154fc30,5290eab1,5348fd84,53c6e11a,562cec5f,587b5989,65a38bf7,7372e1a5,7525289a,76babcde,795e4a37,7ad3efc6,828e68f9,84538528,86ba578b,89aace00,8af75982,90d848e0,Activity,All Star Sorting,Assessment,Bird Measurer (Assessment)_2010,Bird Measurer (Assessment)_3021,Bird Measurer (Assessment)_3121,Bird Measurer (Assessment)_4020,Bottle Filler (Activity)_4070,Bubble Bath_2025,Bubble Bath_2035,Bubble Bath_4020,Bubble Bath_4080,Bug Measurer (Activity)_4035,Cart Balancer (Assessment),Cart Balancer (Assessment)_2000,Cart Balancer (Assessment)_2020,Cart Balancer (Assessment)_3010,Cart Balancer (Assessment)_3110,Cauldron Filler (Assessment),Cauldron Filler (Assessment)_2000,Cauldron Filler (Assessment)_3020,Cauldron Filler (Assessment)_3120,Cauldron Filler (Assessment)_4020,Cauldron Filler (Assessment)_4030,Cauldron Filler (Assessment)_4035,Cauldron Filler (Assessment)_4040,Chest Sorter (Assessment),Chest Sorter (Assessment)_2030,Chest Sorter (Assessment)_3021,Chest Sorter (Assessment)_3121,Chest Sorter (Assessment)_4020,Chest Sorter (Assessment)_4025,Chow Time_4070,Clip,Crystal Caves - Level 2,Crystal Caves - Level 2_2000,Crystal Caves - Level 3,Crystal Caves - Level 3_2000,Dino Drink_3020,Dino Drink_3120,Fireworks (Activity)_4070,Game,Happy Camel_4020,Happy Camel_4030,Happy Camel_4045,Happy Camel_4080,Leaf Leader_2070,Leaf Leader_2075,Leaf Leader_4080,Mushroom Sorter (Assessment)_3010,Mushroom Sorter (Assessment)_3110,Mushroom Sorter (Assessment)_3120,Ordering Spheres,Ordering Spheres_2000,Pan Balance_4080,Sandcastle Builder (Activity),Sandcastle Builder (Activity)_4020,Sandcastle Builder (Activity)_4035,Sandcastle Builder (Activity)_4070,Scrub-A-Dub,Watering Hole (Activity)_2010,a1e4395d,a52b92d5,a7640a16,acc_Bird Measurer (Assessment),acc_Cart Balancer (Assessment),acc_Cauldron Filler (Assessment),acc_Chest Sorter (Assessment),acc_Mushroom Sorter (Assessment),accumulated_accuracy,accumulated_accuracy_group,accumulated_actions,accumulated_uncorrect_attempts,acf5c23f,b120f2ac,c51d8688,c54cf6c5,c58186bf,cf82af56,duration_mean,duration_std,e4d32835,e4f1efe6,e5734469,e694a35b,f6947f54,installation_session_count,session_title,var_event_code,var_event_id,var_title,var_title_event_code
Model,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AdaBoostReg,NaN,5.0,NaN,NaN,NaN,NaN,24.0,26.0,28.0,29.0,30.0,31.0,32.0,48.0,NaN,NaN,33.0,34.0,35.0,41.0,42.0,43.0,44.0,45.0,46.0,47.0,49.0,50.0,NaN,NaN,25.0,27.0,36.0,37.0,39.0,38.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,NaN,20.0,NaN,NaN,NaN,14.0,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,11.0,6.0,22.0,4.0,23.0,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,13.0,NaN,1.0,NaN,NaN,NaN,NaN
CatboostReg,43.0,22.0,33.0,29.0,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0,8.0,11.0,21.0,7.0,10.0,NaN,6.0,NaN,NaN,17.0,12.0,NaN,9.0,31.0,4.0,26.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.0,NaN,NaN,42.0,44.0,NaN,NaN,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.0,NaN,NaN,36.0,NaN,NaN,30.0,NaN,NaN,NaN,NaN,NaN,NaN,48.0,NaN,NaN,NaN,NaN,47.0,NaN,NaN,50.0,NaN,NaN,NaN,NaN,NaN,46.0,NaN,NaN,NaN,NaN,NaN,24.0,NaN,NaN,20.0,1

In [201]:
all_df_pivoted.shape

(7, 169)

In [199]:
fig = px.parallel_coordinates(data_frame=all_df_pivoted,
                              dimensions=all_df_pivoted.columns
                         )

TypeError: object of type 'numpy.int64' has no len()

In [200]:
px.scatter(all_df_pivoted)